# DSCI 512 Lecture 8

Outline:

- Finish up previous class (5 min)
- Caching & memoization (15 min)
- Dynamic programming intro (15 min)
- Break (5 min)
- Dynamic programming: lab 4 (10 min)
- Dynamic programming: backtracking (10 min)
- Code vectorization (20 min)
- Course wrap-up (5 min)

Note: this lecture is too long.
Future self: please solve this by ending lecture 7 on time. thx.

## Learning objectives

- Identify cases of repeated/redundant computation and address them with caching/memoization.
- Describe, at a high level, the basic idea behind dynamic programming.
- Identify problems when dynamic programming is applicable.
- Identify situations where vectorization is possible.
- Convert un-vectorized code to vectorized code.

In [1]:
import functools
import numpy as np

In [2]:
import joblib

In [3]:
import sklearn.metrics

## End of last lecture that we didn't cover (5 min)

## Caching & Memoization (5 min)

- Consider the Fibonacci sequence $0,1,1,2,3,5,8,13,\ldots$.
- The first two numbers are $F_0=0$ and $F_1=1$.
- For the rest, $F_{n}=F_{n-1}+F_{n-2}$; that is, each number is the sum of the previous two numbers.
- How to compute this?

In [4]:
def fib(n):
    f = np.zeros(n+1, dtype=int)
    f[1] = 1
    
    for i in range(2,n+1):
        f[i] = f[i-1] + f[i-2]
    
    return f[-1]

In [5]:
fib(7)

13

Easy. Well, I guess we could also do this recursively?

In [6]:
def fib_rec(n):
    if n == 0 or n == 1:
        return n
    
    return fib_rec(n-1) + fib_rec(n-2)

Now the code looks even cleaner! 

In [7]:
fib_rec(7)

13

But wait...

In [8]:
%timeit -n1 -r1 fib(35)

30.3 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [9]:
%timeit -n1 -r1 fib_rec(35)

3.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


- Hmm... what's the problem?

<br><br><br><br><br><br>

- We can see A LOT of repeated computation. 
- For example, `fib(33)` is called by `fib(34)` and `fib(35)`, and it gets much worse deeper down the tree.
  - Indeed, `fib(1)` is called millions of times...

```
node A --- GIANT GRAPH --- node B --- node C

```

This is a common theme! Other examples:

- Lab 3 `largest_distance` had repeated computations of distances between vertices.
- Factorio optional question from lab 2 recomputes the raw ingredients of an item over and over along different parts of the tree.
- Recursive seam carving code in lab 4 has the same problem.

- How to solve this?
- The general term **caching** refers to storing things for later.
- The specific term **memoiziation** refers to storing the results of function calls for later. See [here](https://stackoverflow.com/questions/6469437/what-is-the-difference-between-caching-and-memoization).
- Confusingly, memoization in Python is called cache; it's OK.

In [10]:
@functools.lru_cache(maxsize=None)
def fib_rec_cache(n):
    if n == 0 or n == 1:
        return n
    
    return fib_rec_cache(n-1) + fib_rec_cache(n-2)

In [11]:
%timeit -n1 -r1 fib_rec_cache(35)

14.4 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


- Amazing! What happened here?
- We ~~haven't~~ have 🎉 learned about [Python decorators](https://realpython.com/primer-on-python-decorators/), but we're remembering the results of calls to `fib_rec`.
  - For example, if `fib_rec(30)` has already been computed, the result ($832040$) is saved and used if `fib_rec(30`) is ever called again.
  - This, we only call `fib_rec(n)` once for each `n`.
- This is a useful trick/concept for speeding things up. 
  - It's a tradeoff of more memory usage for faster running time. 

Other approaches:

- `functools.lru_cache` stored the cache in memory.
- We can use `joblib` to store the cache in a file.

In [12]:
memory = joblib.Memory("/tmp", verbose=0)

In [13]:
@memory.cache
def fib_rec_cache2(n):
    if n == 0 or n == 1:
        return n
    
    return fib_rec_cache2(n-1) + fib_rec_cache2(n-2)



In [14]:
%timeit -n1 -r1 fib_rec_cache2(35)

80.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Dynamic programming: seam carving (25 min)

#### Looking at the lab 

For this part of the class we'll open up lab 4 and take a look at it.
_In the lab, let's go through the intro and the recursive implementation._

```
top of image
  
       x 

bottom of image
```

#### Why is the code so slow?

- This is the same issue as we saw earlier with the Fibonacci numbers.
- We are repeating computations for subproblems.
- We won't do it now, but I encourage you to draw out the recursion and see for yourself!
- We could use the memoization trick again, but:
  - We don't have a clear idea of how much memory it will take - will it be too much?
  - We don't have a clear idea of how long it will run.
  - It's a bit heavy handed and we cannot clearly see what the code is doing.
  - It would be better to just directly write good code for this.

#### Enter dynamic programming 

- Dynamic programming is an algorithm for solving certain optimization problems.
  - It only works for some problems, but when it works it is FAST.
  - This is just like linear programming (last lecture), except for a different set of problems.
  - It is closely related to memoization; see [here](https://stackoverflow.com/questions/6184869/what-is-the-difference-between-memoization-and-dynamic-programming) for a discussion of memoization vs. dynamic programming. 
- Dynamic programming has a lot of applications:
  - DNA sequence alignment
  - Text hyphenation
  - Running certain machine learning models (e.g. hidden Markov models)
  - Finding the differences between two files (bonus - below!)
  - Image resizing (this week's lab!)
  - and more...
- What does the name "Dynamic Programming" mean?
  - "Programming" is often used for optimization, for historical reasons.
  - There is no significance to "Dynamic", it just sounds good.

## Break (5 min)

- Important clarification: dynamic programming != the cache decorator thing.
- I wanted to give you the idea of repeated computation
- To lead into unnecessary computation
- To lead into dynamic programming.

#### And now back to the lab

- Let's look at the code for Exercise 3

In [15]:
def find_vertical_seam(energy):
    nrows, ncols = energy.shape

    # Cumulative Minimum Energy
    CME = np.zeros((nrows, ncols+2))
    CME[:,0] = np.inf
    CME[:,-1] = np.inf
    CME[:,1:-1] = energy

    for i in range(1,nrows):
        for j in range(1,ncols+1):
            CME[i,j] += min(CME[i-1,j-1],CME[i-1,j],CME[i-1,j+1])

    # create the seam array
    seam = np.zeros(nrows, dtype=int)
    seam[-1] = np.argmin(CME[-1,:])

    # track the path backwards
    for i in range(nrows-2,-1,-1):
        # min_index is 0, 1, or 2. I subtract 1 to give the offset from
        # seam(i+1), namely -1, 0, or 1. Then I add this to the old value.
        delta = np.argmin(CME[i, seam[i+1]-1:seam[i+1]+2]) - 1
        seam[i] =  seam[i+1] + delta

    return seam - 1 
    # Above: -1 because the indices are all off by 1, due to padding of CME
    #        This has nothing to do with the -1 in defining "delta'".


In [16]:
np.random.seed(3)
nrows, ncols = 5,5
energy = np.random.randint(1,10, size=(nrows,ncols))
energy

array([[9, 4, 9, 9, 1],
       [6, 4, 6, 8, 7],
       [1, 5, 8, 9, 2],
       [7, 3, 3, 2, 4],
       [6, 9, 2, 9, 8]])

What is the lowest energy seam from the top to the bottom?

In [17]:
seam = find_vertical_seam(energy)
seam

array([1, 1, 0, 1, 2])

- Note that the optimal seam does not include the "1" at the top!
- This illustrates the point that a seam has to be globally optimal, not just locally!

- How does this work?
- We need to compute the cumulative minimum energy from the top to a given location.
- For the top row, it's just the energy values themselves. 

In [18]:
CME = np.zeros((nrows, ncols+2))
CME[:,0] = np.inf
CME[:,-1] = np.inf
CME[:,1:-1] = energy
CME

array([[inf,  9.,  4.,  9.,  9.,  1., inf],
       [inf,  6.,  4.,  6.,  8.,  7., inf],
       [inf,  1.,  5.,  8.,  9.,  2., inf],
       [inf,  7.,  3.,  3.,  2.,  4., inf],
       [inf,  6.,  9.,  2.,  9.,  8., inf]])

- Now, what is the least energy needed to get to position (1,0)? 
  - Well, we could do 9->6 or 4->6
  - 4->6 is better. 
  - So the best we can do is 4+6=10

In [19]:
CME_1_1 = min(CME[0,0], CME[0,1], CME[0,2]) + CME[1,1]
CME_1_1

10.0

- About about for position (1,1)?
  - 4->4 is best

In [20]:
CME_1_2 = min(CME[0,1], CME[0,2], CME[0,3]) + CME[1,2]
CME_1_2

8.0

Etc. For the whole row:

In [21]:
for j in range(1,ncols+1):
    CME[1,j] += min(CME[0,j-1],CME[0,j],CME[0,j+1])
CME

array([[inf,  9.,  4.,  9.,  9.,  1., inf],
       [inf, 10.,  8., 10.,  9.,  8., inf],
       [inf,  1.,  5.,  8.,  9.,  2., inf],
       [inf,  7.,  3.,  3.,  2.,  4., inf],
       [inf,  6.,  9.,  2.,  9.,  8., inf]])

- Ok! So now the 2nd row is done!
- Now we can see why the padding with `np.inf` is useful along the sides:
  - This way we can reuse the same code that takes the minimum of 3 values.
  - We don't have to deal with the edges as special cases.
  - After all, `np.inf` will never be the minimum (all the actual energy values are regular numbers).
  - One could code it up without this but the code is a bit nicer this way.
  - And you'll use this trick again in the last part of the lab.

So, let's do this for the rest of the rows:

In [22]:
for i in range(2,nrows): # in the real code this loop starts from 1, but we already did row 1 manually
    for j in range(1,ncols+1):
        CME[i,j] += min(CME[i-1,j-1],CME[i-1,j],CME[i-1,j+1])
CME

array([[inf,  9.,  4.,  9.,  9.,  1., inf],
       [inf, 10.,  8., 10.,  9.,  8., inf],
       [inf,  9., 13., 16., 17., 10., inf],
       [inf, 16., 12., 16., 12., 14., inf],
       [inf, 18., 21., 14., 21., 20., inf]])

- Part 1 complete.
- Let's take a breath for a moment before continuing.

## Dynamic programming: backtracking (skipped)

- So what we have so far as the minimum possible energy expenditure from the top to a particular location.
- If we now look at the _bottom_ row, we'll see the minimum possible energy _total_.

In [23]:
CME

array([[inf,  9.,  4.,  9.,  9.,  1., inf],
       [inf, 10.,  8., 10.,  9.,  8., inf],
       [inf,  9., 13., 16., 17., 10., inf],
       [inf, 16., 12., 16., 12., 14., inf],
       [inf, 18., 21., 14., 21., 20., inf]])

In [24]:
np.min(CME[-1])

14.0

- Apparently this is 14.
- OK, so now we have the optimal objective function value, 14.
- But we still need the decision variables (seam) that yields this objective value.
- We can retrace our steps here, which is called "backtracking":
  - Start at 14
  - Where could we have come from?
  - Either the 12, the 16, or the other 12.
  - The original energy value of the bottom point was 2:

In [25]:
energy

array([[9, 4, 9, 9, 1],
       [6, 4, 6, 8, 7],
       [1, 5, 8, 9, 2],
       [7, 3, 3, 2, 4],
       [6, 9, 2, 9, 8]])

- So this 14 must have come from $12+2$. 
- In this (unusual) case there are two different 12s, typically it'd be unique.
- What we're seeing here is two seams that are equally good!
- Let's take the left branch.

In [26]:
CME

array([[inf,  9.,  4.,  9.,  9.,  1., inf],
       [inf, 10.,  8., 10.,  9.,  8., inf],
       [inf,  9., 13., 16., 17., 10., inf],
       [inf, 16., 12., 16., 12., 14., inf],
       [inf, 18., 21., 14., 21., 20., inf]])

- So now we're at the 12.
- Above it we see 9, 13, 16.
- We could again look at the energy value there, which happens to be 3, meaning we came from the 9.
- But actually there's an easier way:
  - Where we came from must be the lowest of the three possible CME values (9, 13, 16)
  - Because that is exacly how we computed the CME of 12 in the first place!!
  - So we don't need to look and find that the energy there was 3, we just know it to be the case.
- We continue this process all the way back up:  

In [27]:
seam = np.argmin(CME[-1])
print(seam)

3


In [28]:
np.argmin(CME[-2, seam-1:seam+2]) - 1

-1

This means go left.

In [29]:
seam = seam - 1
np.argmin(CME[-3, seam-1:seam+2]) - 1

-1

This means go left

In [30]:
seam = seam - 1
np.argmin(CME[-4, seam-1:seam+2]) - 1

1

This means go right

In [31]:
seam = seam + 1
np.argmin(CME[-5, seam-1:seam+2]) - 1

0

- This means go up.
- So from the bottom, we started at position 3 of the modified array.
  - This is potition 2 in the original array.
  - left, left, right, up:
  - 2 -> 1 -> 0 -> 1 -> 1
  - But this is backwards so it's [1, 1, 0, 1, 2]
  

In [32]:
find_vertical_seam(energy)

array([1, 1, 0, 1, 2])

Good thing I don't ask you to write the code anymore, I think that was a bit unecessary.

#### When can we use dynamic programming?

- When can we use dynamic programming? 
  - Sequential decision-making: one can make an optimal decision by taking the optimal solution to a subproblem, and looking only at the next step.
- Question: Can we use dynamic programming to solve our MDS TA assignment problem?


<br><br><br>
- Answer: No. If we have assigned 5 TAs optimally, we cannot just assign the 6th TA optimally without looking at the rest of the TA pool. 

## Code Vectorization (20 min)

Consider the code:

In [33]:
n = 10**7
x = np.zeros(n)
y = np.zeros(n)

In [34]:
%%timeit -n1 -r1

for i in range(n):
    y[i] = x[i] + 1

4.03 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Compare with:

In [35]:
y = np.zeros(n)

In [36]:
%%timeit -n1 -r1

y = x + 1

16.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


- Wow, this this ran a lot faster! Why?!
- This is called **vectorization**.
- (Optional) This has to do with the Python language being _interpreted_ rather than _compiled_.
  - You may have worked with other languages, like C or Java, that had a compilation step.
  - Note: R is also interpreted.
- Numpy has some super-optimized code under the hood, for common operations like vector addition
  - Writing `x + 1` tells the interpreter to do vector addition which is fast.
  - Writing the loop has Python execute each step one at a time.
  - It doesn't "know in advance" that it will be a vector addition.
- There are some ways around this, discussed below.
  - There are also some languages that don't suffer from this issue, like [Julia](https://en.wikipedia.org/wiki/Julia_(programming_language)).
- But in general, it's often good to vectorize your code when possible.
- The speedups can be huge. 

#### Case study: pairwise distances

Consider two vectors (1D numpy arrays) $x$ and $y$:

In [37]:
d = 100

x = np.random.rand(d)
y = np.random.rand(d)

The squared Euclidean distance between two vectors $x$ and $y$ is defined as

$$\sum_{i=1}^d (x_i - y_i)^2$$

We definitely don't want to implement this with a loop - we can use `np.sum`:

In [38]:
np.sum((x-y)**2)

15.337953373218195

- Now imagine we have a bunch of $x$ vectors and a bunch of $y$ vectors, and we want to compute the squared distance between all pairs.
  - For example, think of KNN classification where you have a bunch of train vectors and a bunch of test vectors.


In [39]:
nx = 5
ny = 3

X = np.random.rand(nx,d) # we have nx of them
Y = np.random.rand(ny,d) # we have ny of them

Here, we have $5$ $x$-vectors and $3$ $y$-vectors, so we will have $5\times 3 = 15$ distance pairs.

In [40]:
squared_distances = np.zeros((nx,ny))

for i in range(nx):
    for j in range(ny):
        squared_distances[i,j] = np.sum((X[i]-Y[j])**2)
        
squared_distances

array([[16.45044963, 16.57440775, 16.42308959],
       [18.28721673, 15.00038461, 13.02764908],
       [13.08522752, 17.96707571, 16.19017444],
       [15.87051195, 16.95390091, 14.66619765],
       [11.81458429, 18.42825745, 15.15205817]])

Let's wrap everything in a function for timing purposes:

In [41]:
def pairwise_squared_distances(X, Y):
    nx = len(X)
    ny = len(Y)
    
    squared_distances = np.zeros((nx,ny))
    for i in range(nx):
        for j in range(ny):
            squared_distances[i,j] = np.sum((X[i]-Y[j])**2)

    return squared_distances

In [42]:
X = np.random.rand(5, 100) 
Y = np.random.rand(3, 100)

pairwise_squared_distances(X, Y)

array([[18.38917922, 18.07747894, 17.96680562],
       [16.50135296, 15.53137547, 16.33320306],
       [15.40252805, 16.05500748, 17.45413749],
       [16.24033663, 14.38279876, 19.69047128],
       [16.59927659, 15.30817853, 15.75408885]])

In [43]:
Xbig = np.random.rand(1000, 100) 
Ybig = np.random.rand(1000, 100)

In [44]:
%timeit -n1 -r1 pairwise_squared_distances(Xbig, Ybig)

6.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Question: what is the running time of `pairwise_squared_distances`?
<br><br><br><br><br><br>

The running time here is $O(dn_xn_y)$ because of the loops over $n_x$ and $n_y$, and the implicit loop over $d$ inside of `sum`.

#### Broadcasting

- You learned about broadcasting in DSCI 511. 

In [45]:
X.shape

(5, 100)

In [46]:
Y.shape

(3, 100)

In [47]:
X[:,np.newaxis].shape

(5, 1, 100)

In [48]:
Y[np.newaxis].shape

(1, 3, 100)

In [49]:
(X[:,None] - Y[None]).shape

(5, 3, 100)

In [50]:
xx = np.random.rand(5,1)
xx

array([[0.90410728],
       [0.76983655],
       [0.12040406],
       [0.61314058],
       [0.76914491]])

In [51]:
yy = np.random.rand(1,3)
yy

array([[0.47520232, 0.63405477, 0.09001631]])

In [52]:
xx-yy

array([[ 0.42890497,  0.27005252,  0.81409098],
       [ 0.29463423,  0.13578178,  0.67982024],
       [-0.35479825, -0.5136507 ,  0.03038776],
       [ 0.13793826, -0.02091419,  0.52312427],
       [ 0.29394259,  0.13509014,  0.6791286 ]])

In [53]:
(X[:,None] - Y[None]).shape

(5, 3, 100)

In [54]:
(X[:,None] - Y[None])[1,2,3]

-0.2762265262237462

In [55]:
X[1,3] - Y[2,3]

-0.2762265262237462

In [56]:
def pairwise_squared_distances_broadcast(X, Y):    
    return np.sum((X[:,None] - Y[None])**2,axis=2)

In [57]:
pairwise_squared_distances(X, Y)

array([[18.38917922, 18.07747894, 17.96680562],
       [16.50135296, 15.53137547, 16.33320306],
       [15.40252805, 16.05500748, 17.45413749],
       [16.24033663, 14.38279876, 19.69047128],
       [16.59927659, 15.30817853, 15.75408885]])

In [58]:
pairwise_squared_distances_broadcast(X, Y)

array([[18.38917922, 18.07747894, 17.96680562],
       [16.50135296, 15.53137547, 16.33320306],
       [15.40252805, 16.05500748, 17.45413749],
       [16.24033663, 14.38279876, 19.69047128],
       [16.59927659, 15.30817853, 15.75408885]])

In [59]:
%timeit -n1 -r1 pairwise_squared_distances(Xbig, Ybig)

6.42 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [60]:
%timeit -n1 -r1 pairwise_squared_distances_broadcast(Xbig, Ybig)

781 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


- This gives the same result and is around 5x faster!!
- BTW, here is the worst implementation, with no vectorization:

In [61]:
def pairwise_squared_distances_so_slow(X, Y):
    nx = len(X)
    ny = len(Y)
    d = X.shape[1]
    
    squared_distances = np.zeros((nx,ny))
    for i in range(nx):
        for j in range(ny):
            for k in range(d):
                squared_distances[i,j] += (X[i,k]-Y[j,k])**2

    return squared_distances

In [62]:
pairwise_squared_distances_so_slow(X, Y)

array([[18.38917922, 18.07747894, 17.96680562],
       [16.50135296, 15.53137547, 16.33320306],
       [15.40252805, 16.05500748, 17.45413749],
       [16.24033663, 14.38279876, 19.69047128],
       [16.59927659, 15.30817853, 15.75408885]])

In [63]:
# %timeit -n1 -r1 pairwise_squared_distances_so_slow(Xbig, Ybig)

- The above takes around 3 minutes on my laptop, I don't want to run it live.

#### (optional) An even faster approach

- Turns out there's a function that does this:

In [64]:
sklearn.metrics.pairwise.euclidean_distances(X, Y, squared=True)

array([[18.38917922, 18.07747894, 17.96680562],
       [16.50135296, 15.53137547, 16.33320306],
       [15.40252805, 16.05500748, 17.45413749],
       [16.24033663, 14.38279876, 19.69047128],
       [16.59927659, 15.30817853, 15.75408885]])

In [65]:
%timeit -n1 -r1 sklearn.metrics.pairwise.euclidean_distances(Xbig, Ybig, squared=True)

20.9 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


- How is it so fast?! That is almost 100x faster than broadcasting?!
- This is a specific trick to this problem, using this identity:

$$(x-y)^2=x^2+y^2-2xy$$

If $x$ and $y$ are vectors, this becomes

$$(x-y)^T(x-y)=x^Tx+y^Ty-2x^Ty$$

- If we want to do this for all pairs for vectors, we can write it using matrix multiplication $XX^T + YY^T - 2XY^T$
- Matrix multiplication is extremely fast, both because of the implementations and because of a fast algorithm called the [Strassen algorithm](https://en.wikipedia.org/wiki/Strassen_algorithm).

**(end optional)**

#### When should I vectorize my code?

- If you see a loop, ask yourself if the next iteration depends on the result from the previous one.
- If not, try to vectorize it.
- This is not so simple - when summing an array, does each iteration depend on the previous one?
   - Yes, because you keep adding to the cumulative sum.
   - But no, because you could, e.g., recursively sum pairs of numbers.
   - This question is not always that simple, but often you will be able to tell.

#### Faster algorithms vs. faster implementations

- Sometimes we can speed up our code using a faster _algorithm_, like dynamic programming.
  - This usually means that it's actually different in big O, or the number of operations.
- Other times we speed it up using a faster _implementation_, like vectorization.
  - More of these approaches coming below.
- **These are two distinct approaches to speeding up your code.**
  - But sometimes they can be combined, e.g. in the super-fast pairwise distances, or in your lab this week.

This is all we have time for, but there are some bonus materials below on:

- Profiling your code (to tell you what lines are slow)
- Parallelizing code execution
- Other tools to speed up Python
- A really cool (but trickier) dynamic programming example: computing diffs

## Course wrap-up (5 min)

Course learning goals:

- Analyze the scalability and trade-offs of various basic algorithms and data structures using Big-O notation.
- Read and interpret recursive functions.
- Select appropriate data structures, such as graphs, given a data set.
- Map certain real-world problems to discrete optimization problems.
- Diagnose rate-limiting aspects of slow Python code and enumerate various options for speeding it up.

Hopefully this course has helped you more deeply understand some data structures you were already using, like sets, and also unlocked some new ways of thinking, like recursion or optimization.



------------------

Everything below this line is optional / we don't have time for it.

------------------

In [66]:
%load_ext snakeviz

In [69]:
from numba import njit, prange

In [72]:
from sklearn.neighbors import KNeighborsClassifier

## (optional) Profiling (5 min)

- Profiling means measuring how long parts of your code takes.
- We'll use [SnakeViz](https://jiffyclub.github.io/snakeviz/) for this.

In [73]:
knn = KNeighborsClassifier(algorithm='brute')
knn.fit(Xbig, np.zeros(Xbig.shape[0]));

In [74]:
%%snakeviz -t 

# Note: -t is to open in new tab which is necessary in JupyterLab (thanks to Joel for this tip)

knn.predict(Ybig)

 
*** Profile stats marshalled to file '/var/folders/sl/_n6p71ds1vb5tr2zmd73gnpr0000gp/T/tmp7_r80qv2'. 
Opening SnakeViz in a new tab...


- Here, we can see that a lot of time was spent in the `pairwise_distances_chunked` function.
- On the other hand, if we remove `algorithm='brute'` we'll see the code runs faster and we'll also see different profile results.

- Profiling is useful when your code is slow but you don't know which parts.
- "Premature optimization is the root of all evil" -Donald Knuth
  - So, make sure you know what part of your code needs speeding up.
  - If something is only taking 1% of your code's total time, don't work on making it faster.

## (optional) Other tools for speeding up Python code (5 min)

#### Numba

 
- "[Numba](http://numba.pydata.org/) is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code."
  - "JIT compiler" = Just-in-time compiler.
  - It sees a loop and figures out that it's actually vector addition, right before the loop is executed (hence "just in time").
  - This is the easiest to deploy, because you don't need to change your code - just add a decorator.

Recall that this code took about 7 seconds last time:

In [75]:
n = 10**7
x = np.zeros(n)
y = np.zeros(n)

def run_code(x, y):
    for i in range(len(x)):
        y[i] = x[i] + 1

In [76]:
%timeit -n1 -r1 run_code(x, y)

3.59 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [77]:
@njit
def run_code_jit(n, x, y):
    for i in range(n):
        y[i] = x[i] + 1

In [78]:
%timeit -n1 -r1 run_code_jit(n, x, y)

397 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


- Bam! It's just that easy.
- Note: with the `@` we're again using a decorator, like we did with the caching at the start of class.
- This is rather confusing because `@` can also denote matrix multiplication...

This is amazing - why isn't it the default?

- This goes back to the idea that Python wasn't designed for numerical computation. 
- This is a 3rd party install
- In this case we could have also vectorized our code instead.

#### Cython

- [Cython](https://cython.org/) allows the user to leverage the increased speed of C within Python.
- You can call C routines.
- You actually compile your Cython code - it is not interpreted.

#### PyPy

- [PyPy](https://pypy.org/) is an implementation of Python in Python.
  - The original Python, or CPython, is written in C.
  - PyPy has several advantages including a just-in-time compiler which might make your code run faster.
- You can install PyPy and run your code with `pypy test.py` instead of `python test.py`.
  - You don't need to change your code, BUT some libraries may not work with PyPy. Things like numpy do work. For more info, see [here](https://pypy.org/compat.html).

Note! This has nothing to do with [PyPI](https://pypi.org/), which is the package index, like [CRAN](https://cran.r-project.org/) for R. 

## (optional) Parallelization (5 min)

- [Moore's law](https://en.wikipedia.org/wiki/Moore%27s_law) states that computing power (number of transistors on a chip) will double approximately every 2 years. 
  - Shockingly, this held true for ~40 years.
  - Recently, progress has tapered off.
  - However, recently progress has been in **parallel** and **distributed** computation.

Consider our Fibonacci code from earlier:

In [79]:
def fib(n):
    f = np.zeros(n+1, dtype=int)
    f[1] = 1
    
    for i in range(2,n+1):
        f[i] = f[i-1] + f[i-2]
    
    return f[-1]

- The running time here is $O(n)$.
- Now consider our addition code:


In [80]:
def add_one(n, x):
    y = np.zeros(n)
    for i in range(n):
        y[i] = x[i] + 1
        
    return y

- The running time here is also $O(n)$.
- But this one can be done in parallel, because the $n$ computations _do not depend on each other_.
- On the other hand, each Fibonacci iteration depends on the previous two values.
- Both are $O(n)$, but they are fundamentally different.
- The requirements for code to be parallelized and vectorized are similar, though the two are not the same thing.

#### Hardware for parallelization


In [81]:
import multiprocessing

multiprocessing.cpu_count()

8

- My laptop has 8 cores. 
- I should theoretically be able to use them to run the code in parallel and achieve (close to) a 8x speedup.
- Recently, [GPUs](https://en.wikipedia.org/wiki/Graphics_processing_unit) have become extremely popular for parallel computing, especially in deep learning.
  - We'll talk about this in DSCI 572.

#### Software for parallelization

- Numba can do some of this:

In [82]:
n = 50_000_000

x = np.zeros(n)
y = np.zeros(n)

@njit
def run_code(x, y):
    for i in range(len(x)):
        y[i] = x[i] + np.random.rand()

In [83]:
%timeit -n1 -r1 run_code(x, y)

951 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [84]:
@njit(parallel=True)
def run_code_parallel(x, y):
    for i in prange(len(x)): # <- note the prange instead of range
        y[i] = x[i] + np.random.rand()

In [85]:
%timeit -n1 -r1 run_code_parallel(x, y)

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


787 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


- In this case the speedup is modest (I don't know the details), but it can be more.
- Even a 2x speedup is great if your code takes 4 hours instead of 8 hours.

- We saw [joblib](https://joblib.readthedocs.io/en/latest/) earlier, for on-disk memoization.
- It can also be used for parallelization, but we won't go into it here.

#### Parallel vs. distributed

- Parallel computing can happen even on one machine, e.g. my 4 cores
- Distributed computing means sharing the computation across machines.
  - We'll talk about this in DSCI 525.

Parallel computing is an entire field, so we don't do it much justice in 5 min. But this was hopefully better than nothing.

## Bonus dynamic programming application: diffs between strings

- When you look at a commit in GitHub, you just see the "diff" or difference between the two versions.

![](diff.png)

- Note the highlighted areas: git/GitHub knows about the added whitespace, changing "see" to "SEE", and the deleted exclamation mark.
  - How does GitHub compute the diff?
  - We want to highlight _characters that are not present in the other version_.
  - (Optional note: I believe it's GitHub, rather than git, that's doing this. See [here](https://stackoverflow.com/questions/10398744/does-git-store-diff-information-in-commit-objects).)
- Why is this an optimization problem?
  - You can always say the diff is "deleted the entire old file, added the entire new file".
  - But that is not a useful diff!
  - What we are doing implicitly is _aligning_ the two sequences.
  - We want an alignment with the _minimal_ changes that go from $x$ to $y$, i.e. we want to _minimize the number of highlighted characters._
  - Now we have an optimization problem!
- We have 4 highlighted characters in $x$ (red) and 6 highlighted characters in $y$ (green), for a total of 10.

Back to the example:

![](diff_cropped.png)


#### We're really _aligning_ the two strings

- When we compare the strings and see a mismatch, how do we know if we should highlight in red or green?
  - What we're really doing here is an _alignment_; it has to be the green.
  - We have a binary decision to make everytime we hit a mismatch.
- But we can't just do this one character at a time: the highlighting affects the alignment!
  - We need to look globally at the whole of strings.
- We want something that is optimal across _all possible alignments_.
  - This brings us to the brute force approach: recursively check all possible alignments. 
- Optional note: this is a reformulation of the [longest common subsequence problem](https://en.wikipedia.org/wiki/Longest_common_subsequence_problem) and is also a form of [edit distance](https://en.wikipedia.org/wiki/Edit_distance).

In [86]:
def num_diffs_rec(x, y):
    """
    Find the number of characters in the diff between x and y.
    
    Parameters
    ----------
    x : str
        The first string
    y : str
        The second string
        
    Returns
    -------
    int
        The number of highlights
        
    Examples
    --------
    >>> num_diffs_rec("This is a!", "this  is a")
    4
    >>> num_diffs_rec("xxHello", "Hellox")
    3
    """    
    if len(x) == 0:
        return len(y) # Highlight the rest of y
    if len(y) == 0:
        return len(x) # Highlight the rest of x
    
    if x[0] == y[0]:  # A match
        return num_diffs_rec(x[1:], y[1:]) 
    else:
        return 1 + min( num_diffs_rec(x[1:], y), num_diffs_rec(x, y[1:]) )

In [87]:
num_diffs_rec("This is a!", "this  is a")

4

In [88]:
num_diffs_rec("xxHello", "Hellox")

3

Note: the code that actually returns the highlighted characters is slightly more complicated -- we'll defer that until later.

#### How does this work?

Back to the example:

```
This is a demonstration of diffs in git/GitHub. Let's see if it works!
This    is a demonstration of diffs in git/GitHub. Let's SEE if it works
```

- The first mismatch is between `i` and ` `. 
- Remember, we're looking for a _minimum_ number of highlights.
- The minimum total number of highlights is the lesser of two things:
   1. The score if you highlight `i`, plus 1 for highlighting the `i`.
   2. The score if you highlight ` `, plus 1 for highlighting the ` `.
- So we do this recursively and that's it!
- The base case is that if you reach the end of one string, you have to highlight the rest of the other string.

#### How slow is this code?

- As usual, the brute force solution is unusably slow.

In [89]:
%timeit -n1 -r1 num_diffs_rec("This'll be slow", "Yeah, right...")

30.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


This implementation is pretty much useless!

#### Why is the code so slow?

- This is the same issue as we saw earlier with the Fibonacci numbers.
- We are repeating computations for subproblems.
- We won't do it now, but I encourage you to draw out the recursion and see for yourself!
- We could use the memoization trick again, but:
  - We don't have a clear idea of how much memory it will take - will it be too much?
  - We don't have a clear idea of how long it will run.
  - It's a bit heavy handed and we cannot clearly see what the code is doing.
  - It would be better to just directly write good code for this.

#### Enter dynamic programming 

- Dynamic programming is an algorithm for solving certain optimization problems.
  - It only works for some problems, but when it works it is FAST.
  - This is just like linear programming (last lecture), except for a different set of problems.
  - It is closely related to memoization; see [here](https://stackoverflow.com/questions/6184869/what-is-the-difference-between-memoization-and-dynamic-programming) for a discussion of memoization vs. dynamic programming. 
- Dynamic programming has a lot of applications:
  - DNA sequence alignment
  - Text hyphenation
  - Running certain machine learning models (e.g. hidden Markov models)
  - Finding the differences between two files (this!)
  - Image resizing (this week's lab!)
  - and more...
- What does the name "Dynamic Programming" mean?
  - "Programming" is often used for optimization, for historical reasons.
  - There is no significance to "Dynamic", it just sounds good.

#### Dynamic programming implementation

In [90]:
def num_diffs(x, y, return_table=False):
    """
    Compute the number of highlighted characters in the
    diff between x and y using dynamic programming.
    
    Parameters
    ----------
    x : str
        The first string
    y : str
        The second string
        
    Returns
    -------
    numpy.ndarray
        The dynamic programming table. 
        The last element is the result.
        
    Examples
    --------
    >>> num_diffs("This is a!", "this  is a")[-1,-1]
    4
    >>> num_diffs("xxHello", "Hellox")[-1,-1]
    3
    """   
    M = len(x)
    N = len(y)
    
    opt = np.zeros((M+1, N+1), dtype=int)
    opt[:,0] = np.arange(M+1)
    opt[0,:] = np.arange(N+1)
    
    for i in range(1,M+1):
        for j in range(1,N+1):
            if x[i-1] == y[j-1]:
                opt[i,j] = opt[i-1, j-1]
            else:
                opt[i,j] = 1 + min( opt[i-1,j], opt[i,j-1] )

    return opt if return_table else opt[-1,-1]

In [91]:
num_diffs("This is a!", "this  is a")

4

In [92]:
num_diffs("xxHello", "Hellox")

3

#### How does this work?

- The recursive implementation works "top-down":
  - It starts from the big problem (the entire two strings) and computes the solutions for subproblems.
- Dynamic programming works "bottom-up":
  - It starts from the smallest subproblems and builds to the bigger ones.
  - This way it doesn't recompute things multiple times.
  - It is basically the recursive solution + memoization, but implemented deliberately.
  
To be concrete:

- We defined a 2D array which we called `opt`. 
- `opt[i,j]` contains `num_diffs(x[:i], y[:j])`, that is the solution for the $(i,j)$ subproblem
- The key logic is as follows:
  - If `x[i]` equals `y[j]`, then `opt[i,j]` equals `opt[i-1,j-1]`
    - Because we don't add any additional highlights by adding that next character!
  - If they are not equal, then there are two possibilities: highlight in $x$ or highlight in $y$
    - We want the **better** of those two possibilities.
    - Good news is, we already know how good they are! This is the subproblem part.
  - This is how we build up the solution iteratively instead of repeating computation.

In [93]:
x, y = "xxHello", "Hellox"

opt = num_diffs(x, y, return_table=True)
opt

array([[0, 1, 2, 3, 4, 5, 6],
       [1, 2, 3, 4, 5, 6, 5],
       [2, 3, 4, 5, 6, 7, 6],
       [3, 2, 3, 4, 5, 6, 7],
       [4, 3, 2, 3, 4, 5, 6],
       [5, 4, 3, 2, 3, 4, 5],
       [6, 5, 4, 3, 2, 3, 4],
       [7, 6, 5, 4, 3, 2, 3]])

Let's recreate this table according to its definition:

In [94]:
same = 0*opt
for i in range(len(x)+1):
    for j in range(len(y)+1):
        same[i,j] = num_diffs_rec(x[:i], y[:j])
same

array([[0, 1, 2, 3, 4, 5, 6],
       [1, 2, 3, 4, 5, 6, 5],
       [2, 3, 4, 5, 6, 7, 6],
       [3, 2, 3, 4, 5, 6, 7],
       [4, 3, 2, 3, 4, 5, 6],
       [5, 4, 3, 2, 3, 4, 5],
       [6, 5, 4, 3, 2, 3, 4],
       [7, 6, 5, 4, 3, 2, 3]])

- This illustrates how silly the recursive solution is!!
- Each element of this table `same` was computed from scratch.
- But actually we can get it in $O(1)$ time given the previous elements.
  - That's how the dynamic programming code works.

#### Computational cost

- Now we know how much it costs to compute this:
  - Memory usage is $O(MN)$
  - Runtime is also $O(MN)$
  - (where $M$ is the length of $x$ and $N$ is the length of $y$)

#### Recovering the highlights

- This is called "backtracking" in dynamic programming.
- We need access the table and make sense of it.

In [95]:
opt

array([[0, 1, 2, 3, 4, 5, 6],
       [1, 2, 3, 4, 5, 6, 5],
       [2, 3, 4, 5, 6, 7, 6],
       [3, 2, 3, 4, 5, 6, 7],
       [4, 3, 2, 3, 4, 5, 6],
       [5, 4, 3, 2, 3, 4, 5],
       [6, 5, 4, 3, 2, 3, 4],
       [7, 6, 5, 4, 3, 2, 3]])

- Intuitively, we check where we might have come from at the previous step.
- We start at the lower-right and "backtrack" to the upper left.
- The function below has some extra HTML visualization code that you can ignore.

In [96]:
from IPython.display import HTML, display

dark_green  = '<span style="background-color: rgba(0,255,0,0.5)">'
dark_red    = '<span style="background-color: rgba(255,0,0,0.5)">'
light_green = '<span style="background-color: rgba(0,255,0,0.05)">'
light_red   = '<span style="background-color: rgba(255,0,0,0.05)">'

def show_diff(x, y, align=False):
    opt = num_diffs(x, y, return_table=True)
    
    x_highlight = ''
    y_highlight = ''
    i = len(x) 
    j = len(y)
    while i > 0 or j > 0:
        if i > 0 and j > 0 and x[i-1] == y[j-1]:
            x_highlight = x[i-1] + x_highlight
            y_highlight = y[j-1] + y_highlight
            i -= 1
            j -= 1
        elif j > 0 and opt[i, j] == opt[i, j-1] + 1:
            y_highlight = dark_green + y[j-1] + '</span>' + y_highlight 
            if align:
                x_highlight = " " + x_highlight
            j -= 1
        else:
            x_highlight = dark_red   + x[i-1] + '</span>' + x_highlight 
            if align:
                y_highlight = " " + y_highlight
            i -= 1
    
    x_highlight = light_red   + x_highlight + "</span>"
    y_highlight = light_green + y_highlight + "</span>"
    
    display(HTML('<code>' + x_highlight + '</code>' + '<br>' + '<code>' + y_highlight + '</code>'))

In [97]:
show_diff(x, y)

(Note: the colours do not render properly on GitHub - you need to run the notebook locally to see the highlighting.)

In [98]:
before = "This is a demonstration of diffs in git/GitHub. Let's see if it works!"
after  = "This    is a demonstration of diffs in git/GitHub. Let's SEE if it works"

show_diff(before, after)

<br><br>vs.<br><br>

![](diff_cropped.png)

- They match! 
- Well, with a slight difference on highlighting of the spaces.
- This is because the solution to the optimization problem is not unique - both are optimal.
- Arbitrary choices made in the code will determine which one is returned by the code.

- Why could we use dynamic programaming to find the diffs between documents?
  - This comes back to the discussion of how `opt` is computed.
  - The optimal solution at a given step can be described in terms of optimal solutions of subproblems.

In [99]:
x, y = "xxHello", "Hellox"

opt = num_diffs(x, y, return_table=True)
opt

array([[0, 1, 2, 3, 4, 5, 6],
       [1, 2, 3, 4, 5, 6, 5],
       [2, 3, 4, 5, 6, 7, 6],
       [3, 2, 3, 4, 5, 6, 7],
       [4, 3, 2, 3, 4, 5, 6],
       [5, 4, 3, 2, 3, 4, 5],
       [6, 5, 4, 3, 2, 3, 4],
       [7, 6, 5, 4, 3, 2, 3]])

Consider `opt[3,1]`

In [100]:
x[:3]

'xxH'

In [101]:
y[:1]

'H'

Here, because the last letters match (both `H`), we can just grab the score from the diagonal entry:

In [102]:
opt[3,1]

2

In [103]:
opt[2,0]

2

Now consider `opt[5,2]`

In [104]:
x[:5]

'xxHel'

In [105]:
y[:2]

'He'

We know this is the right alignment:

In [106]:
show_diff(x[:5], y[:2])

But how do we get there? 

- We need to look at two options: the `l` is highlighted or the `e` is highlighted.
- Since they don't match, one of them has to be highlighted.
- What's confusing here is that, in the final result, neither is highlighted.
  - But for this subproblem, one of them has to be!
  - This "line of reasoning" isn't going to end up being used in the final result at all.
  - But we still have to compute it because we don't know in advance what will end up being optimal.

In [107]:
show_diff(x[:4], y[:2])

In [108]:
opt[4,2]

2

In [109]:
show_diff(x[:5], y[:1])

In [110]:
opt[5,1]

4

In [111]:
opt[5,2]

3